In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from medscraper import medsearch

In [2]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')

In [3]:
url="https://www.medicinpriser.dk/default.aspx?lng=2"

In [8]:
driver=webdriver.Chrome(options=options)
driver.get(url)
driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").send_keys("Latanoprost")
driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)
combo = driver.find_element(By.ID, "ctl00_ctl07_Results_PagerTop_ResultsCountList")
combo_select = Select(combo)
combo_select.select_by_visible_text("All")

In [9]:
#Identificar la tabla que cuenta con estas características "<table cellpadding="0" border="0" cellspacing="0" style="width:100%">

table_element = driver.find_element(By.XPATH, "//table[@cellpadding='0' and @border='0' and @cellspacing='0' and @style='width:100%']")
table_contents = table_element.get_attribute('outerHTML')
soup = BeautifulSoup(table_contents, 'html.parser')

In [26]:
def tabla_dinamarca():
    table_element = driver.find_element(By.XPATH, "//table[@cellpadding='0' and @border='0' and @cellspacing='0' and @style='width:100%']")
    table_contents = table_element.get_attribute('outerHTML')
    return table_contents


In [32]:
driver=webdriver.Chrome(options=options)
driver.get(url)
try:
    driver.find_element(By.ID, "ctl00_ctl07_simpleForm_VirksomtstofBox").send_keys("nexavar")
    driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)
    try: 
        tabla=tabla_dinamarca(driver)
    except NoSuchElementException:
        driver.find_element(By.ID, "ctl00_ctl07_simpleForm_LaegemiddelBox").send_keys("nexavar")
        driver.find_element(By.ID, "ctl00_ctl07_simpleForm_SearchButton").send_keys(Keys.ENTER)
        try:
            tabla=tabla_dinamarca(driver)
            except NoSuchElementException:
            print("No se encontró el elemento")






TypeError: tabla_dinamarca() takes 0 positional arguments but 1 was given

In [10]:
producto=[]
presentacion=[]
precio=[]
for i in soup.find_all('div', {'class':'width282'}):
    j=i.find_all('table', {'class':'grid'})


In [11]:
tableprod = soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridStatic'})
tableprecio=soup.find('table', attrs={'id': 'ctl00_ctl07_Results_ResultGridScroll'})


In [12]:
#obtener rows de las dos tablas
rowsprod = tableprod.find_all('tr')
rowsprecio=tableprecio.find_all('tr')

In [13]:
# And then we can loop through the rows to extract the data
caract=[]

for row in rowsprod:
    columns = row.find_all('td')
    for column in columns:
        #Append el texto con saltos de 2 en 2
        caract.append(column.text)

producto=caract[::3]
producto=[re.sub(r'\d+', '', i) for i in producto]
presentacion=caract[1::3]
paquete=caract[2::3]

caract2=[]
for row in rowsprecio:
    columns = row.find_all('td')
    for column in columns:
        #Append el texto con saltos de 2 en 2
        caract2.append(column.text)

#Obtener precio en múltiplos de 5
precio=caract2[4::5]
#Sustituir coma por punto
precio=[re.sub(r',', '.', i) for i in precio]


In [14]:
def euro():

    exchangerates='https://openexchangerates.org/api/latest.json?app_id=52adcacdba9c48ce8430667723c02968&base=USD&symbols=MXN,EUR'
    r=requests.get(exchangerates).json()
    #Obtener tipos de cambio
    euro=r['rates']['MXN']/r['rates']['EUR']
    return euro


In [15]:
df=pd.DataFrame({'producto':producto, 'presenta':presentacion, 'paquete':paquete, 'precio':precio})
df[['presentacion','cantidad']]=df.presenta.str.split(" ",expand=True)
df=df[~df.precio.str.contains("Discont")]
df["precio"]=df["precio"].astype(float)
df["precio"]=round(df["precio"]*euro(),2)
df

,producto,presenta,paquete,precio,presentacion,cantidad
0,Nexavar,200 mg,112 stk. (blister)filmovertrukne ta ...,66283.96,200,mg
3,"Sorafenib ""Mylan""",200 mg,112 stk. (blister)filmovertrukne ta ...,62756.20,200,mg


In [18]:
s=medsearch.scraper("fsanpablo","sorafenib")
mex=s.show_dataframe()
mex

Realizando la búsqueda en Farmacias San Pablo de:  sorafenib


,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod
0,nexavar 112 comprimidos caja tesilato de soraf...,73287.0,20:06:16 23/06/2023,fsanpablo,sorafenib,farmacias san pablo,Mexico,nexavar,200.0,mg,tableta,112


In [20]:
print("Diferencial de precio entre México y Dinamarca: ",mex["precio"]-df["precio"][0], " pesos")

Diferencial de precio entre México y Dinamarca:  0    7003.04
Name: precio, dtype: float64  pesos
